In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import pickle
import gc
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings


In [2]:
df_shop=pd.read_csv("data//shop_info.csv")
df_behavior=pd.read_csv("data//user_shop_behavior.csv")
X_train=df_behavior.merge(df_shop[['shop_id','mall_id']],on=['shop_id'],how='left')
X_train['row_id']=X_train.index

In [3]:
X_test=pd.read_csv("data//test.csv")
wifi_train=pd.read_pickle("data//wifi_train.pkl")

In [4]:
K=6#rssi最短间距

shop_bssid(店铺能收到哪些wifi)：{shop_id:[bssid_id]}<br>
bssid_shop(wifi在哪些店铺出现过)：{bssid_id:[shop_id]}<br>
mall_shop(商场有哪些店铺)：{mall_id:[shop_id]}<br>

In [6]:
wifi_train=wifi_train.merge(X_train[['row_id','shop_id']],on='row_id',how='left')
shop_bssid=wifi_train.groupby('shop_id',as_index=False)['bssid'].agg({'all_bssid':'unique'})
bssid_shop=wifi_train.groupby('bssid',as_index=False)['shop_id'].agg({'all_shop':'unique'})
mall_shop=wifi_train.groupby('mall_id',as_index=False)['shop_id'].agg({'all_shop':'unique'})
shop_bssid=dict(shop_bssid.values)
bssid_shop=dict(bssid_shop.values)
mall_shop=dict(mall_shop.values)

构建指纹库

In [7]:
#建窗口指纹库
for num,(day1,day2) in enumerate([(1,17),(15,31)]):
    fingerbase=dict()
    X_train_=X_train.loc[(X_train['day']>=day1)&(X_train['day']<=day2),:]
    for index,row in tqdm(X_train_.iterrows(),total=X_train_.shape[0]):
        wifi_list=dict()
        for wifi in row.wifi_infos.split(';'):
            info=wifi.split('|')
            wifi_list[info[0]]=int(info[1])

        finger=dict()
        for bssid1 in wifi_list:
            for bssid2 in wifi_list:
                if wifi_list[bssid1]-wifi_list[bssid2]>=K:
                    if bssid1 not in finger:
                        finger[bssid1]=set()
                    finger[bssid1].add(bssid2)

        if row['shop_id'] not in fingerbase:
            fingerbase[row['shop_id'] ] = dict()
        for key in finger:
            if key not in fingerbase[row['shop_id']]:
                fingerbase[row['shop_id']][key]=set()
            for value in finger[key]:
                fingerbase[row['shop_id']][key].add(value)

    pkl_file=open('data//fingerbase//fingerbase_6K_%s_%s.pkl'% (day1,day2),'wb')
    pickle.dump(fingerbase,pkl_file)

100%|██████████| 646701/646701 [04:24<00:00, 2447.83it/s]


匹配指纹库得分<br><br>
res结构:{row_id:{shop_id:score}}<br>
cnt为dict, 记录该条数据wifi 在该商场各shop出现次数+1

In [8]:
#窗口指纹库计算得分

for num,(day1,day2) in enumerate([(18,31)]):
    res=dict()
    fingerbase=pd.read_pickle('data//fingerbase//fingerbase_6K_1_17.pkl')
    X_=X_train.loc[(X_train['day']>=day1)&(X_train['day']<=day2),:]

    for index,row in tqdm(X_.iterrows(),total=X_.shape[0]):#修改
        wifi_list=dict()
        cnt = dict()
        for wifi in row.wifi_infos.split(';'):
            info=wifi.split('|')
            wifi_list[info[0]]=int(info[1])
            if info[0] in bssid_shop:
                for shop_id in bssid_shop[info[0]]:
                    if shop_id not in cnt:
                        cnt[shop_id]=0
                    cnt[shop_id]+=1
        for shop_id in mall_shop[row['mall_id']]:
            if shop_id not in cnt:
                cnt[shop_id]=0
            cnt[shop_id]+=1

        finger=dict()
        for bssid1 in wifi_list:
            for bssid2 in wifi_list:
                if wifi_list[bssid1]-wifi_list[bssid2]>=K:
                    if bssid1 not in finger:
                        finger[bssid1]=set()
                    finger[bssid1].add(bssid2)

        if row['row_id'] not in res:
            res[row['row_id']]=dict()
        for shop_id in mall_shop[row['mall_id']]:
            score = 0
            for key in finger.keys():
                if shop_id not in fingerbase:
                    continue
                if key not in fingerbase[shop_id]:
                    continue
                for value in finger[key]:
                    if value in fingerbase[shop_id][key]:
                        score+=1
            res[row['row_id']][shop_id]=score

    #存储得分
    pkl_file=open('data//fingerbase//train_score_6K_%s_%s_.pkl'%(day1,day2),'wb')
    pickle.dump(res,pkl_file)

100%|██████████| 556289/556289 [11:22<00:00, 814.71it/s]


In [9]:
res=dict()
fingerbase=pd.read_pickle('data//fingerbase//fingerbase_6K_15_31.pkl')
X_=X_test.copy()

for index,row in tqdm(X_.iterrows(),total=X_.shape[0]):#修改
    wifi_list=dict()
    cnt = dict()
    for wifi in row.wifi_infos.split(';'):
        info=wifi.split('|')
        wifi_list[info[0]]=int(info[1])
        if info[0] in bssid_shop:
            for shop_id in bssid_shop[info[0]]:
                if shop_id not in cnt:
                    cnt[shop_id]=0
                cnt[shop_id]+=1
    for shop_id in mall_shop[row['mall_id']]:
        if shop_id not in cnt:
            cnt[shop_id]=0
        cnt[shop_id]+=1

    finger=dict()
    for bssid1 in wifi_list:
        for bssid2 in wifi_list:
            if wifi_list[bssid1]-wifi_list[bssid2]>=K:
                if bssid1 not in finger:
                    finger[bssid1]=set()
                finger[bssid1].add(bssid2)

    if row['row_id'] not in res:
        res[row['row_id']]=dict()
    for shop_id in mall_shop[row['mall_id']]:
        score = 0
        for key in finger.keys():
            if shop_id not in fingerbase:
                continue
            if key not in fingerbase[shop_id]:
                continue
            for value in finger[key]:
                if value in fingerbase[shop_id][key]:
                    score+=1
        res[row['row_id']][shop_id]=score

#存储得分
pkl_file=open('data//fingerbase//test_score_6K_twoweek_.pkl','wb')
pickle.dump(res,pkl_file)


100%|██████████| 483931/483931 [10:12<00:00, 790.44it/s]
